<a href="https://colab.research.google.com/github/MadmanMarble/Planet_U-NET/blob/main/AOI_make_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages(c('raster','sf','fasterize'))
install.packages("googledrive")
install.packages("devtools")

devtools:::install_github("gearslaboratory/gdalUtils")
install.packages("dplyr")

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages(c("raster", "sf", "fasterize")):
“installation of package ‘raster’ had non-zero exit status”


In [ ]:
library(raster)
library(sf)
library(fasterize)
library(magrittr)
library(googledrive)
library(parallel)
library(data.table)
library(stringr)
library(dplyr)
# library(gdalUtils)

drive_auth(use_oob = TRUE, cache = FALSE)

In [ ]:
dir.create("/content/labels")
dir.create("/content/imagery")
dir.create("/content/imagery/planet")
dir.create("/content/imagery/planet/2018")
dir.create("/content/imagery/planet/2019")
dir.create("/content/imagery/planet/2020")
dir.create("/content/workingfolder")
dir.create("/content/workingfolder/aoi")
dir.create("/content/workingfolder/dem")
dir.create("/content/workingfolder/ccap")
dir.create ("/content/workingfolder/nlcd")

## load images and assign variables

In [ ]:
# Load Planet 2018 utm4 select files
# https://drive.google.com/drive/folders/1EX6fe0vObp8VOLcGqxFsxIZhQ9mu0un_?usp=drive_link
#
jp_folder = "1EX6fe0vObp8VOLcGqxFsxIZhQ9mu0un_"
folder_id = drive_get(as_id(jp_folder)) # get folder ID
print(folder_id) #print folder ID

# Find files in the folder
files = drive_ls(folder_id)

# Loop through directories and download files inside them
for (i in seq_along(files$name)) {
  i_dir <- files[i, ] # Select directory
    try({

      # Download files from the directory to the specified path
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/imagery/planet/2018/', i_dir$name[1])
      )
    })
  }

# Load Planet 2019 utm4 select files
# https://drive.google.com/drive/folders/1h-OSRbTn3AhbNswTb3jooqRs68QB92T2?usp=drive_link
jp_folder = "1h-OSRbTn3AhbNswTb3jooqRs68QB92T2"
folder_id = drive_get(as_id(jp_folder)) # get folder ID
print(folder_id) #print folder ID

# Find files in the folder
files = drive_ls(folder_id)

# Loop through directories and download files inside them
for (i in seq_along(files$name)) {
  i_dir <- files[i, ] # Select directory
    try({

      # Download files from the directory to the specified path
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/imagery/planet/2019/', i_dir$name[1])
      )
    })
  }

# Load Planet 2020 utm4 select files
# https://drive.google.com/drive/folders/1sUlR77kvCtXjXPy7z2nztJzTvhUvXU9w?usp=drive_link
jp_folder = "1sUlR77kvCtXjXPy7z2nztJzTvhUvXU9w"
folder_id = drive_get(as_id(jp_folder)) # get folder ID
print(folder_id) #print folder ID

# Find files in the folder
files = drive_ls(folder_id)

# Loop through directories and download files inside them
for (i in seq_along(files$name)) {
  i_dir <- files[i, ] # Select directory
    try({

      # Download files from the directory to the specified path
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/imagery/planet/2020/', i_dir$name[1])
      )
    })
  }



In [ ]:
vrt_utm4_2018 <- stack("/content/imagery/planet/2018/input_tif_2018.vrt")
vrt_utm4_2018 %>% plot
vrt_utm4_2019 <- stack("/content/imagery/planet/2019/input_tif_2019.vrt")
vrt_utm4_2019 %>% plot
vrt_utm4_2020 <- stack("/content/imagery/planet/2020/input_tif_2020.vrt")
vrt_utm4_2020 %>% plot

## load working folder and assign variables

In [ ]:
# https://drive.google.com/drive/folders/1SJtFMy9oe2D8dFrWoMgRmLXL2N6oNvej?usp=drive_link
jp_folder = "1SJtFMy9oe2D8dFrWoMgRmLXL2N6oNvej"
folder_id = drive_get(as_id(jp_folder)) # get folder ID
print(folder_id) #print folder ID

# Find files in the folder
files = drive_ls(folder_id)

# Loop through directories and download files inside them
for (i in seq_along(files$name)) {
  i_dir <- files[i, ] # Select directory
    try({

      # Download files from the directory to the specified path
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/workingfolder/aoi/', i_dir$name[1])
      )
    })
  }


In [ ]:
Haleiwa <- st_read("/content/workingfolder/aoi/Haleiwa.gpkg")
Haleiwa %>% plot
nc <- st_read("/content/workingfolder/aoi/Haleiwa.gpkg")
nc %>% plot

In [ ]:
Kahuku <- st_read("/content/workingfolder/aoi/Kahuku.gpkg")
Kahuku %>% plot

In [ ]:
KawainuiMarsh <- st_read("/content/workingfolder/aoi/KawainuiMarsh.gpkg")
KawainuiMarsh %>% plot

In [ ]:
PearlHarbor <- st_read("/content/workingfolder/aoi/PearlHarbor.gpkg")
PearlHarbor %>% plot

In [ ]:
aoi_extent_384 <- st_read("/content/workingfolder/aoi/aoi_extent_384.gpkg")
aoi_extent_384 %>% plot

In [ ]:
oahu_aoi_outline <- st_read("/content/workingfolder/aoi/oahu_aoi_outline.gpkg")
oahu_aoi_outline %>% plot

In [ ]:
# nc = st_read(system.file("shape/nc.shp", package="sf"))
#> Reading layer `nc' from data source
#>   `/home/runner/work/_temp/Library/sf/shape/nc.shp' using driver `ESRI Shapefile'
#> Simple feature collection with 100 features and 14 fields
#> Geometry type: MULTIPOLYGON
#> Dimension:     XY
#> Bounding box:  xmin: -84.32385 ymin: 33.88199 xmax: -75.45698 ymax: 36.58965
#> Geodetic CRS:  NAD27
g = st_make_grid(nc)
plot(g)
plot(st_geometry(nc), add = TRUE)
# g[nc] selects cells that intersect with nc:
plot(g[nc], col = '#ff000088', add = TRUE)

In [ ]:
g_intersect = st_as_sf(g[nc])
g_intersect %>% plot

In [ ]:
# Save the intersecting grid as a .gpkg file
st_write(g_intersect, "/content/sample_data/intersecting_grid.gpkg", driver = "GPKG")

In [ ]:
g = st_make_grid(nc)
plot(g)
plot(st_geometry(nc), add = TRUE)
# g[nc] selects cells that intersect with nc:
plot(g[nc], col = '#ff000088', add = TRUE)